In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as skl_train_test_split
from surprise import Reader, Dataset, SVD, KNNBasic, NormalPredictor, BaselineOnly, NMF
from surprise.model_selection import cross_validate, KFold
from collections import defaultdict

plt.rcParams['font.family'] = 'Times New Roman'

# Preparacion de datos

In [3]:
df_ratings = pd.read_csv('TP2/datos/rating_final.csv')
df_places = pd.read_csv('TP2/datos/geoplaces2.csv')

datasets_locales = ['TP2/datos/chefmozaccepts.csv', 'TP2/datos/chefmozcuisine.csv', 'TP2/datos/chefmozhours4.csv', 'TP2/datos/chefmozparking.csv']

for dataset in datasets_locales:
    df_temp = pd.read_csv(dataset)
    merged_places = pd.merge(df_places, df_temp, on='placeID', how='left')

print(f"Shape del DataFrame original de locales: {merged_places.shape}")

merged_places = df_places.drop_duplicates(subset=['placeID'])

merged_places.to_csv('TP2/datos/dataLocal.csv', sep=';', index=False)

for dataset in datasets_locales:
    df_temp = pd.read_csv(dataset)
    df_places = pd.merge(df_places, df_temp, on='placeID', how='left')

avg_ratings = df_ratings.groupby('placeID')[['rating', 'food_rating', 'service_rating']].mean().reset_index()

df_places = pd.merge(df_places, avg_ratings, on='placeID', how='left')
df_places = df_places.drop_duplicates(subset=['placeID'])
df_places.to_csv('TP2/datos/dataLocal_with_averages.csv', sep=';', index=False)

print(f"Shape del DataFrame final de locales con promedios: {df_places.shape}")

Shape del DataFrame original de locales: (130, 22)
Shape del DataFrame final de locales con promedios: (130, 29)


In [3]:
userprofile_file = 'TP2/datos/userprofile.csv'
usercuisine_file = 'TP2/datos/usercuisine.csv'
userpayment_file = 'TP2/datos/userpayment.csv'

df_userprofile = pd.read_csv(userprofile_file, sep=',')
df_usercuisine = pd.read_csv(usercuisine_file, sep=',')
df_userpayment = pd.read_csv(userpayment_file, sep=',')

df_users = pd.merge(df_userprofile, df_usercuisine, on='userID', how='left')
df_users = pd.merge(df_users, df_userpayment, on='userID', how='left')

df_users = df_users.drop_duplicates(subset=['userID'])

avg_user_ratings = df_ratings.groupby('userID')[['rating', 'food_rating', 'service_rating']].mean().reset_index()

df_users = pd.merge(df_users, avg_user_ratings, on='userID', how='left')

df_users.to_csv('TP2/datos/dataUser.csv', sep=';', index=False)

print(f"Shape del DataFrame original de usuarios: {df_users.shape}")

Shape del DataFrame original de usuarios: (138, 24)


In [87]:
ratings = df_users
locales = df_places

In [88]:
print(ratings.columns)
print(locales.columns)

Index(['userID', 'latitude', 'longitude', 'smoker', 'drink_level',
       'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos',
       'birth_year', 'interest', 'personality', 'religion', 'activity',
       'color', 'weight', 'budget', 'height', 'Rcuisine', 'Upayment', 'rating',
       'food_rating', 'service_rating'],
      dtype='object')
Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'Rpayment', 'Rcuisine', 'hours', 'days',
       'parking_lot', 'rating', 'food_rating', 'service_rating'],
      dtype='object')


# Similitud del coseno

In [4]:
df_ratings = pd.read_csv('TP2/datos/dataRatings.csv', delimiter=';')

df_ratings = df_ratings.drop_duplicates(subset=['userID', 'placeID'])

df_ratings.to_csv('TP2/datos/dataRatings.csv', index=False, sep=';')

In [90]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

categorical_features = ['alcohol', 'smoking_area', 'dress_code', 'accessibility', 'price', 'Rambience', 'franchise', 'area', 'other_services', 'Rpayment', 'Rcuisine']
numerical_features = ['latitude', 'longitude', 'rating', 'food_rating', 'service_rating']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
locales_preprocessed = pipeline.fit_transform(locales)

similarity_matrix = cosine_similarity(locales_preprocessed, locales_preprocessed)

print(similarity_matrix)

[[1.         0.29052311 0.26067155 ... 0.43389833 0.7050241  0.17912732]
 [0.29052311 1.         0.62682767 ... 0.13611863 0.49574583 0.57941364]
 [0.26067155 0.62682767 1.         ... 0.22696956 0.15812585 0.33536408]
 ...
 [0.43389833 0.13611863 0.22696956 ... 1.         0.51621685 0.15246621]
 [0.7050241  0.49574583 0.15812585 ... 0.51621685 1.         0.43144178]
 [0.17912732 0.57941364 0.33536408 ... 0.15246621 0.43144178 1.        ]]


In [91]:
def recommend_locales_based_on_similarity(local_input, n=15):
    try:
        local_index = locales.loc[locales.name == local_input].index[0]
    except IndexError:
        return "Local no encontrado. Asegúrate de que el nombre sea correcto."

    similarity_score = list(enumerate(similarity_matrix[local_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    similarity_score = similarity_score[1:n+1]
    local_indices = [i[0] for i in similarity_score]

    return locales[['name', 'address', 'city', 'rating', 'food_rating', 'service_rating']].iloc[local_indices]

recomendaciones = recommend_locales_based_on_similarity('carnitas_mata', n=15)
print(recomendaciones)

                                    name  \
522            Hamburguesas Valle Dorado   
639                    Hamburguesas saul   
57             Abondance Restaurante Bar   
645                      Rincon Huasteco   
669                         Cafe Chaires   
705                 Restaurant Orizatlan   
898                     sirloin stockade   
330                            Luna Cafe   
441                           Potzocalli   
354                          el pueblito   
261      Restaurante la Estrella de Dima   
465  Restaurante El Chivero S.A. de C.V.   
531                          dairy queen   
54                        Restaurante 75   
3                        puesto de tacos   

                                      address             city    rating  \
522                                 Av. Coral  San Luis Potosi  0.800000   
639  Av. Saan Luis enttre moctezuma y salinas  San Luis Potosi  0.600000   
57                Industrias 908 Valle Dorado  San Luis Potosi  0.5

# SR usando Surprise

In [92]:
print(df_ratings)

     userID  placeID  rating  food_rating  service_rating   latitude  \
0     U1077   135085       2            2               2  22.150802   
1     U1077   135038       2            2               1  22.155651   
2     U1077   132825       2            2               2  22.147392   
3     U1077   135060       1            2               2  22.156883   
4     U1068   135104       1            1               2  23.752982   
...     ...      ...     ...          ...             ...        ...   
1156  U1043   132630       1            1               1  23.752931   
1157  U1011   132715       1            1               0  23.732423   
1158  U1068   132733       1            1               0  23.752707   
1159  U1068   132594       1            1               1  23.752168   
1160  U1068   132660       0            0               0  23.752943   

       longitude                                     the_geom_meter  \
0    -100.982680  0101000020957F00009F823DA6094858C18A2D4D37F9A4

In [93]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(0, 2))
data = Dataset.load_from_df(df_ratings[['userID', 'placeID', 'rating']], reader)

In [94]:
from surprise import SVD
from surprise.model_selection import cross_validate

svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7360  0.7209  0.7010  0.7193  0.0143  
MAE (testset)     0.6345  0.6118  0.6032  0.6165  0.0132  
Fit time          0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.73599195, 0.72092471, 0.701023  ]),
 'test_mae': array([0.6344627 , 0.61184337, 0.60324751]),
 'fit_time': (0.004285097122192383,
  0.002206087112426758,
  0.002485036849975586),
 'test_time': (0.0021359920501708984,
  0.0008480548858642578,
  0.0010077953338623047)}

In [95]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [96]:
svd.predict(uid=10, iid=100)

Prediction(uid=10, iid=100, r_ui=None, est=1.1998277347114557, details={'was_impossible': False})

In [97]:
print(df_ratings)

     userID  placeID  rating  food_rating  service_rating   latitude  \
0     U1077   135085       2            2               2  22.150802   
1     U1077   135038       2            2               1  22.155651   
2     U1077   132825       2            2               2  22.147392   
3     U1077   135060       1            2               2  22.156883   
4     U1068   135104       1            1               2  23.752982   
...     ...      ...     ...          ...             ...        ...   
1156  U1043   132630       1            1               1  23.752931   
1157  U1011   132715       1            1               0  23.732423   
1158  U1068   132733       1            1               0  23.752707   
1159  U1068   132594       1            1               1  23.752168   
1160  U1068   132660       0            0               0  23.752943   

       longitude                                     the_geom_meter  \
0    -100.982680  0101000020957F00009F823DA6094858C18A2D4D37F9A4

In [98]:
df_ratings.loc[df_ratings.name.str.contains('Tortas Locas Hipocampo'), :]

,userID,placeID,rating,food_rating,service_rating,latitude,longitude,the_geom_meter,name,address,...,url,Rambience,franchise,area,other_services,Rpayment,Rcuisine,hours,days,parking_lot
0,U1077,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
68,U1108,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
211,U1081,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
246,U1056,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
257,U1134,135085,2,1,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
347,U1116,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
403,U1132,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
420,U1120,135085,0,0,0,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
450,U1036,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public
485,U1084,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,?,familiar,f,closed,none,cash,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;,public


# Generando recomendaciones

In [99]:
import difflib
import random

def get_local_id(nombre_local, metadata):
    
    existing_names = list(metadata['name'].values)
    closest_names = difflib.get_close_matches(nombre_local, existing_names)
    place_id = metadata[metadata['name'] == closest_names[0]]['placeID'].values[0]
    return place_id

def get_local_info(local_id, metadata):
    
    local_info = metadata[metadata['placeID'] == local_id][['id', 'latitude', 
                                                    'longitude', 'address']]
    return local_info.to_dict(orient='records')

def predict_review(userID, local_name, model, metadata):
    
    placeID = get_local_id(local_name, metadata)
    review_prediction = model.predict(uid=userID, iid=placeID)
    return review_prediction.est

def generate_recommendation(userID, model, metadata, thresh=4):
    
    local_names = list(metadata['name'].values)
    random.shuffle(local_names)
    
    for local_name in local_names:
        rating = predict_review(userID, local_name, model, metadata)
        if rating >= thresh:
            placeID = get_local_id(local_name, metadata)
            return get_local_info(placeID, metadata)

In [100]:
generate_recommendation('U1103', svd, locales, 4.8)

# TSNE

In [101]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, n_iter=500, verbose=3, random_state=1)
locales_embedding = tsne.fit_transform(svd.qi)
projection = pd.DataFrame(columns=['x', 'y'], data=locales_embedding)
projection['nombreGenerado'] = locales['name']

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 130 samples in 0.000s...
[t-SNE] Computed neighbors for 130 samples in 0.011s...
[t-SNE] Computed conditional probabilities for sample 130 / 130
[t-SNE] Mean sigma: 0.334978
[t-SNE] Computed conditional probabilities in 0.002s
[t-SNE] Iteration 50: error = 59.3879623, gradient norm = 0.8302994 (50 iterations in 0.275s)
[t-SNE] Iteration 100: error = 64.8134003, gradient norm = 0.8059993 (50 iterations in 0.027s)
[t-SNE] Iteration 150: error = 65.0180817, gradient norm = 0.7475141 (50 iterations in 0.022s)
[t-SNE] Iteration 200: error = 72.0568237, gradient norm = 0.3900565 (50 iterations in 0.022s)
[t-SNE] Iteration 250: error = 69.4488373, gradient norm = 0.4997140 (50 iterations in 0.022s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.448837
[t-SNE] Iteration 300: error = 1.2112625, gradient norm = 0.0143107 (50 iterations in 0.024s)
[t-SNE] Iteration 350: error = 1.0809973, gradient norm = 0.0079607 (

In [102]:
import plotly.express as px
import datapane as dp

fig = px.scatter(
    projection, x='x', y='y'
)
fig.show()

report = dp.Report(dp.Plot(fig) )

In [103]:
from collections import defaultdict
from surprise import Dataset, SVD

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [104]:
algo = SVD()
algo.fit(trainset)

In [105]:
locales_visitados_usuario=df_ratings.loc[(df_ratings.userID=='U1077'),'placeID']
locales_no_visitados_usuario=df_ratings.loc[~df_ratings.placeID.isin(locales_visitados_usuario),'placeID'].unique()

pred=[]
for local in locales_no_visitados_usuario:
    pred.append(svd.predict(uid='U1077', iid=local))

In [106]:
pred

[Prediction(uid='U1077', iid=135104, r_ui=None, est=1.0628600223403257, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132740, r_ui=None, est=1, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132663, r_ui=None, est=1.1461664954115671, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132732, r_ui=None, est=1.1183741234200022, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132630, r_ui=None, est=1.1980552045778083, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132584, r_ui=None, est=1.21046678253609, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132733, r_ui=None, est=1.189309447333155, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132560, r_ui=None, est=1.2849437208098795, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132613, r_ui=None, est=1.223008100891107, details={'was_impossible': False}),
 Prediction(uid='U1077', iid=132667, r_ui=None, e

In [107]:
top_n = get_top_n(pred, n=10)
top_n

defaultdict(list,
            {'U1077': [(134986, 1.5404549383791382),
              (135034, 1.5158396007381667),
              (132955, 1.5002341839263467),
              (135025, 1.4927151063070279),
              (134996, 1.457453110955359),
              (135013, 1.4362897401918227),
              (135028, 1.4305165934116821),
              (135059, 1.4258781377250753),
              (132768, 1.425251246637),
              (132754, 1.4121672146033586)]})

In [108]:
for uid, user_ratings in top_n.items():
    print([locales.loc[locales.placeID==iid,'name'].values[0] for (iid, _) in user_ratings])

['Restaurant Las Mananitas', 'Michiko Restaurant Japones', 'emilianos', 'El Rincon de San Francisco', 'Sanborns Casa Piedra', 'Giovannis', 'La Virreina', 'Restaurant Bar Hacienda los Martinez', 'Mariscos Tia Licha', 'Cabana Huasteca']


# Comparacion desempeño algoritmos

In [164]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import GridSearchCV, cross_validate, KFold
from surprise.prediction_algorithms import NormalPredictor, BaselineOnly, SVD, KNNBasic, NMF
from collections import defaultdict
from sklearn.model_selection import train_test_split as sklearn_train_test_split

ratings_data_f = df_ratings.sample(n=100000, replace=True)
ratings_data_f = ratings_data_f[['userID', 'placeID', 'rating', 'food_rating', 'service_rating']]
reader = Reader(rating_scale=(0, 2))

def evaluate(data_train, algo):
    cv_results = cross_validate(algo, data_train, measures=['RMSE'], cv=4, verbose=False, n_jobs=-1)
    return round(cv_results['test_rmse'].mean(), 3)

def precision_recall_at_k(predictions, k=10, threshold=2.0):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    return precisions, recalls

def evaluate_precision_recall(data_train, algo):
    kf = KFold(n_splits=4)
    precision = []
    recall = []
    for trainset, testset in kf.split(data_train):
        algo.fit(trainset)
        predictions = algo.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=2.0)
        precision.append(sum(prec for prec in precisions.values()) / len(precisions))
        recall.append(sum(rec for rec in recalls.values()) / len(recalls))
    return sum(precision) / len(precision), sum(recall) / len(recall)

algorithms = {
    'Random': (NormalPredictor, {}),
    'Baseline': (BaselineOnly, {}),
    'SVD': (SVD, {'n_factors': [50, 100], 'reg_all': [0.02, 0.05]}),
    'KNN_cosine': (KNNBasic, {'k': [20, 30], 'sim_options': {'name': ['cosine'], 'user_based': [False]}}),
    'KNN_msd': (KNNBasic, {'k': [20, 30], 'sim_options': {'name': ['msd'], 'user_based': [False]}})
}

results_list = []

def train_test_split_custom(df, column, reader):
    train, test = sklearn_train_test_split(df[['userID', 'placeID', column]], test_size=0.20, random_state=100)
    data_train = Dataset.load_from_df(train, reader)
    return data_train, test

for column in ['rating', 'food_rating', 'service_rating']:
    print(f'Evaluating {column}...')
    data_train, test = train_test_split_custom(ratings_data_f, column, reader)

    for algo_name, (algo_class, param_grid) in algorithms.items():
        print(f'Tuning {algo_name}...')
        try:
            gs = GridSearchCV(algo_class, param_grid, measures=['rmse'], cv=4)
            gs.fit(data_train)

            best_algo = gs.best_estimator['rmse']

            rmse = evaluate(data_train, best_algo)
            precision, recall = evaluate_precision_recall(data_train, best_algo)

            results_list.append({
                'Algoritmo': algo_name,
                'RMSE CV': rmse,
                'Precision': precision,
                'Recall': recall,
                'Rating Type': column
            })
        except ZeroDivisionError as e:
            print(f'Error tuning {algo_name}: {e}')

results_df = pd.DataFrame(results_list)
print(results_df)

Evaluating rating...
Tuning Random...
Tuning Baseline...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Tuning SVD...
Tuning KNN_cosine...
Computing the cosine similarity matrix...
Error tuning KNN_cosine: float division
Tuning KNN_msd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix